In [6]:
import os

import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import display

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import urllib.request
from PIL import Image
import numpy as np

@ interact(season_year=widgets.IntSlider(min=2016, max=2020, step=1, value=1, description='Year'), 
           season=widgets.Dropdown(options=['regular', 'playoffs'], value='regular',description='Season'))
def interactive_tool(season_year: int, season: str):
      
    dir_path = os.path.join(os.environ['NHL_DATA_DIR'], str(season_year), season)
    
    if season == 'regular':
        game_numbers = list(range(1, len(os.listdir(dir_path))+1))
    else: 
        game_numbers = []
        ronde = 4
        matchup = 8
        game = 7
        for i in range(1, ronde + 1):
            for j in range(1, int(matchup) + 1):
                for k in range(1, game + 1):
                    code = int(f'{i}{j}{k}')
                    game_numbers.append(code)
            matchup /= 2

    @ interact(game_id=widgets.SelectionSlider(options=game_numbers, value=115, description='Game ID'))
    def load_json(game_id: int):                   
        game_id = str(game_id).zfill(4) 
        if season == 'regular':
            code = '02'
        else:
            code = '03' 
        file_path = os.path.join(dir_path, f'{season_year}{code}{game_id}.json')
        print(file_path)
        with open(file_path, 'r') as json_file:
              data = json.load(json_file)
        print(data['gameData']['datetime']['dateTime'])
        print(f'Game ID: {game_id}; \n')
        
        print('{}{}{}'.format(''.ljust(15), 'Home'.ljust(7), 'Away'.ljust(7)))
        
        abb_home = data['gameData']['teams']['home']['abbreviation']
        abb_away = data['gameData']['teams']['away']['abbreviation']
        print('{}{}{}'.format('Teams:'.ljust(15), abb_home.ljust(7), abb_away.ljust(7)))
        
        goals_home = str(data['liveData']['plays']['allPlays'][data['liveData']['plays']['scoringPlays'][-1]]['about']['goals']['home'])
        goals_away = str(data['liveData']['plays']['allPlays'][data['liveData']['plays']['scoringPlays'][-1]]['about']['goals']['away'])
        print('{}{}{}'.format('Goals:'.ljust(15), goals_home.ljust(7), goals_away.ljust(7)))
        
        
        print('{}{}{}'.format('SoG:'.ljust(15), ''.ljust(7), ''.ljust(7)))
        print('{}{}{}'.format('50 Goals:'.ljust(15), ''.ljust(7), ''.ljust(7)))
        print('{}{}{}'.format('50 Attempts:'.ljust(15), ''.ljust(7), ''.ljust(7)))
        
        @ interact(event_idx=widgets.IntSlider(min=0, max=max(0, len(data['liveData']['plays']['allPlays'])-1), step=1, value=0, description='Event Index'))
        def print_info(event_idx: int):
            try:
                plt.figure(figsize = (10,10))
                img = mpimg.imread("https://raw.githubusercontent.com/udem-ift6758/project-template/main/figures/nhl_rink.png")
                plt.imshow(img, extent=[-100.0, 100.0, -42.5, 42.5])
                             
                x_coord = data['liveData']['plays']['allPlays'][event_idx]['coordinates']['x']
                y_coord = data['liveData']['plays']['allPlays'][event_idx]['coordinates']['y']
                plt.plot(x_coord, y_coord, marker='o', color='blue', markersize=20)
                
                plt.xlabel('feet')
                plt.ylabel('feet')
                
                title1 = data['liveData']['plays']['allPlays'][event_idx]['result']['description'] 
                title2_0 = data['liveData']['plays']['allPlays'][event_idx]['about']['periodTime']
                title2_1 = data['liveData']['plays']['allPlays'][event_idx]['about']['period']
                title3_0 = data['gameData']['teams']['away']['abbreviation']
                title3_1 = data['gameData']['teams']['home']['abbreviation']
                plt.title(f'{title1}\n{title2_0} P-{title2_1}\n{title3_0}                                                         {title3_1}' )
                
                plt.show()
                print(json.dumps(data['liveData']['plays']['allPlays'][event_idx], indent=4))
            except:
                print(f'No data for this event')

interactive(children=(IntSlider(value=2016, description='Year', max=2020, min=2016), Dropdown(description='Sea…